### **RAG를 이용한 설문지 생성**

In [41]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
import re
import win32com.client as win32
import os
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_groq import ChatGroq

In [ ]:
# hwp2pdf
def hwp2pdf(folder_path, sav_path):
    
    # 한글 기본 설정 
    hwp = win32.gencache.EnsureDispatch("hwpframe.hwpobject")
    hwp.XHwpWindows.Item(0).Visible = False
    hwp.RegisterModule("FilePathCheckDLL", "FilePathCheckerModule")

    #파일 경로
    source_folder = os.path.abspath(folder_path)
    files = [f for f in os.listdir(source_folder) if re.match('.*[.]hwp', f)]
    
    print(f'hwp file {len(files)}개 변환 시작')
    
    for file in files:
        hwp.Open(os.path.join(source_folder, file))  # 파일 열기

        # 저장 경로
        filename = os.path.abspath(os.path.join(sav_path, file).replace('hwp', 'pdf'))
        print(filename)
        
        # PDF 저장 
        act = hwp.CreateAction("Print")
        pset = act.CreateSet()
        act.GetDefault(pset)
        pset.SetItem("FileName", filename)  
        pset.SetItem("PrinterName", "Microsoft Print to PDF") 
        pset.SetItem("PrintMethod", 0)   
        act.Execute(pset)
        
        hwp.Run("FileClose")  
        
        print(f'{file} 변환 완료')

    hwp.Quit() 

source_folder = './data/survey/hwp'
save_folder = './data/survey/pdf'

hwp2pdf(source_folder, save_folder)

In [ ]:
# 문서 로드 
path = r"C:\Users\bigdata\Desktop\새 폴더\[STI]_(설문지)_(사)21세기산업연구소_거창군 치유산업 종합계획 수립을 위한 설문조사(관광객 니즈 분석).pdf"
loader = PyMuPDFLoader(path)

docs = loader.load()

In [ ]:
# 텍스트 분리
splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
split_docs = splitter.split_documents(docs)

In [ ]:
# 임베딩 모델 생성
embedder = OpenAIEmbeddings(model = 'text-embedding-3-small')

# vector store 생성
vector_store = FAISS.from_documents(documents = split_docs, embedding=embedder)

# Retreiver 정의
retreiver = vector_store.as_retriever()

In [ ]:
# 프롬프트를 생성
intro_prompt = PromptTemplate.from_template(
    """

# 주제:
{question}

- 설문자에 들어가는 소개말을 작성해줘 예시 포맷을 참고하여 작성해 주세요.

예시 포맷 (참고용):
안녕하십니까?
바쁘신 가운데에서도 설문조사에 참여해 주셔서 감사합니다.본 설문은 ___________ 대상으로 기술 보유 여부 및 현황, 시장성 등 국가전략기술 연구역량을 갖춘 기업체 발굴과 기술 수준을 파악하고자 실시합니다.
 설문에 응답하신 자료는 익명으로 처리되며, 통계법 제33조(비밀의 보호) 및 제34조(통계종사자 등의 의무)에 따라 철저히 비밀이 보장되어 순수 분석 목적 외에 어떤 용도로도 사용하지 않을 것을 약속드립니다. 
응답하신 내용은 ____________에 귀중한 자료로 이용될 것입니다. 
설문조사에 적극 협조해 주신 데 대해 다시 한번 감사드립니다. 

# Answer:

"""
)

llm = ChatGroq(model = 'qwen-qwq-32b')

intro_chain = intro_prompt | llm



In [ ]:
# 프롬프트를 생성
survey_prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer in Korean.

# 주제:
{question}

위 주제에 관한 설문지를 아래 예시 포맷을 참고하여 작성해 주세요.

# 작성 조건:
- 설문지는 총 10문항으로 구성되어야 합니다.
- 다음 유형의 문항을 모두 포함해야 합니다:
  - 만족도 문항 (척도형): 예) 1문항 이상
  - 객관식 문항 (단수 선택): 예) 1문항 이상
  - 객관식 문항 (복수 응답): 예) 1문항 이상
  - 주관식 문항: 예) 1문항 이상
- 각 문항은 "Q번호." 형식으로 시작하며, 선택지가 필요한 경우 각 선택지는 번호로 표시합니다.
- 예시 설명 중 “만족도(척도형), 객관식문항(단수 선택) 등” 은 최종 출력에 포함되지 않아야 합니다.


예시 포맷 (참고용):
-------------------------------
만족도(척도형): 
Q1. 당신은 이 서비스에 얼마나 만족하십니까?
1. 매우 불만족  
2. 불만족  
3. 보통  
4. 만족  
5. 매우 만족  

객관식문항(단수 선택):
Q2. 귀하의 여행 스타일은 무엇입니까?
1. 국내여행  
2. 해외여행  

객관식문항(복수응답):
Q3. 다음 중 여행지 선택 시 가장 중요하게 생각하는 요소는 무엇입니까? (복수 선택 가능)
1. 자연환경  
2. 문화유산  
3. 먹거리  
4. 쇼핑  
5. 기타  

주관식문항:
Q4. 이번 프로그램에 대한 전반적인 의견이나 개선점을 자유롭게 적어 주세요.
-------------------------------

# Context:
{context}

# Answer:"""
)

survey_chain = (
    {'context' : retreiver, 'question' : RunnablePassthrough()}
    | survey_prompt
    | llm
)


In [98]:
combined_chain = RunnableParallel(
    {
        'intro' : intro_chain,
        'survey' : survey_chain
    }
) | RunnableLambda(lambda x: x['intro'].content + '\n' + x['survey'].content)


result = combined_chain.invoke("거창군을 방문하는 관광객을 대상으로 설문조사를 실시하려고 합니다.")

In [99]:
import re

def clean_output(text):
    # 1. <think> 블럭 제거
    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)

    # 2. Q10 이후 설명 제거 (Q10 문항은 남김)
    text = re.sub(r'(Q10\..*?)(\n\n|\n)(.*)', r'\1', text, flags=re.DOTALL)

    return text.strip()

if 'think' in result:
    cleaned_result = clean_output(result)
    
else:
    cleaned_result = result
    
print(cleaned_result)

안녕하십니까?  
바쁘신 가운데에서도 설문조사에 참여해 주셔서 감사합니다. 본 설문은 **거창군을 방문한 관광객**을 대상으로 관광 만족도, 주요 관광지 이용 경험, 추천 콘텐츠 등을 파악하여 지역 관광 서비스 및 시설 개선에 참고할 귀한 의견을 수집하기 위해 실시합니다.  
설문에 응답하신 자료는 익명으로 처리되며, 통계법 제33조(비밀의 보호) 및 제34조(통계종사자 등의 의무)에 따라 철저히 비밀이 보장되어 순수 분석 목적 외에 어떤 용도로도 사용하지 않을 것을 약속드립니다.  
응답하신 내용은 **거창군 관광 발전과 방문객 체험 질적 향상**에 귀중한 자료로 이용될 것입니다.  
설문조사에 적극 협조해 주신 데 대해 다시 한번 감사드립니다.



Q1. 거창군의 자연환경에 대해 얼마나 만족하시나요?  
1. 매우 불만족  
2. 불만족  
3. 보통  
4. 만족  
5. 매우 만족  

Q2. 거창군을 방문한 주된 동기는 무엇인가요?  
1. 자연경관 탐방  
2. 문화체험  
3. 치유 프로그램 참여  
4. 쇼핑  
5. 음식 체험  

Q3. 거창군 관광지 중 방문한 경험이 있는 곳은 어디인가요? (복수 선택 가능)  
1. 감악산 별바람언덕  
2. 가조온천  
3. 사과테마파크  
4. 월성우주창의과학관  
5. 항노화힐링랜드  
6. 수승대  
7. 기타  

Q4. 거창군 치유산업 프로그램 중 가장 관심 있는 것은 무엇인가요?  
1. 자연 치유 프로그램  
2. 문화 치유 프로그램  
3. 체험형 웰니스 프로그램  
4. 전통 치유 문화 체험  
5. 기타  

Q5. 거창군 교통편의성에 대해 어떻게 평가하시나요?  
1. 매우 불편  
2. 불편  
3. 보통  
4. 편리  
5. 매우 편리  

Q6. 거창군 관광지 홍보 매체를 통해 접한 경험이 있는 곳은 어디인가요? (복수 선택 가능)  
1. 지역광고  
2. 인터넷 검색  
3. SNS  
4. 여행사 안내  
5. 지인 추천  
6. 관광홍보誌  
7. 기타  

Q7. 거